In [1]:
import pandas as pd
from brainflow import BoardShim, BrainFlowInputParams
import time
from scipy import signal
import numpy as np
from scipy.signal import welch, butter, lfilter
import pickle
from statistics import mean
import matplotlib.pyplot as plt
from PIL import Image
from pythonosc import udp_client
import random
import warnings
from sklearn.exceptions import DataConversionWarning

In [2]:
# Define the function to compute PSD for multiple frequency bands
def compute_psd_bands(data, fs):
    f, psd = welch(data, fs=fs, nperseg=256)
    
    # Define the frequency ranges for each band
    bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 30),
        'Gamma': (30, 100)
    }
    
    # Compute the PSD for each frequency band
    psd_bands = {}
    for band, (f_min, f_max) in bands.items():
        idx = np.where((f >= f_min) & (f < f_max))[0]
        psd_bands[band] = np.mean(psd[idx])
    
    return psd_bands



In [3]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

In [4]:
# definir high, low, med
def emocion(valence, arousal, dominance):
    if valence == 0:
        if arousal == 0:
            if dominance == 0:
                image = Image.open('img\Sadness_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Sadness'
                
            elif dominance == 1:
                return 'Other'

            elif dominance == 2:
                image = Image.open('img\Rejection_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Rejected'

        elif arousal == 1:
            if dominance == 0:
                return 'Other'

            elif dominance == 1:
                return 'Other'

            elif dominance == 2:
                image = Image.open('img\Calm_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Calm'

        elif arousal == 2:
            if dominance == 0:
                image = Image.open('img\Relief_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Relief'

            elif dominance == 1:
                image = Image.open('img\Relaxed_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Relaxed'

            elif dominance == 2:
                image = Image.open('img\Overconfidence_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Overconfident'         
    if valence == 1:
        if arousal == 0:
            if dominance == 0:
                return 'Other'

            elif dominance == 1:
                return 'Other'

            elif dominance == 2:
                image = Image.open('img\Pessimism_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Pessimistic'

        elif arousal == 1:
            if dominance == 0:
                return 'Other'
                
            elif dominance == 1:
                image = Image.open(r'img\Neutral_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Neutral'

            elif dominance == 2:
                return 'Other'

        elif arousal == 2:
            if dominance == 0:
                image = Image.open('img\Satisfaction_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Sattisfied'

            elif dominance == 1:
                return 'Other'

            elif dominance == 2:
                return 'Other' 

    if valence == 2:
        if arousal == 0:
            if dominance == 0:
                image = Image.open('img\Hate_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Hate'

            elif dominance == 1:
                image = Image.open('img\Distressed_'+ str(random.randint(1, 4))+'.jpeg')
                plt.imshow(image)
                return 'Distressed'

            elif dominance == 2:
                image = Image.open('img\Anxiety_'+ str(random.randint(1, 4))+'.jfif')
                plt.imshow(image)
                return 'Anxious'

        elif arousal == 1:
            if dominance == 0:
                image = Image.open('img\Admiration_'+ str(random.randint(1, 4))+'.jpeg')
                plt.imshow(image)
                return 'Admiration'

            elif dominance == 1:
                image = Image.open('img\Desire_'+ str(random.randint(1, 7))+'.jpeg')
                plt.imshow(image)
                return 'Desire'

            elif dominance == 2:
                return 'Other'

        elif arousal == 2:
            if dominance == 0:
                image = Image.open('img\Love_'+ str(random.randint(1, 7))+'.jpeg')
                plt.imshow(image)
                return 'Love'

            elif dominance == 1:
                image = Image.open('img\Joy_'+ str(random.randint(1, 4))+'.jpeg')
                plt.imshow(image)
                return 'Joy'

            elif dominance == 2:
                image = Image.open('img\Generosity_'+ str(random.randint(1, 4))+'.jpeg')
                plt.imshow(image)
                return 'Generosity'   

In [5]:
Val_Pkl = pickle.load(open('Val_RF.pkl', 'rb'))
Aro_Pkl = pickle.load(open('Aro_RF.pkl', 'rb'))
Dom_Pkl = pickle.load(open('Dom_RF.pkl', 'rb'))

In [6]:
iteraciones = 0
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
warnings.filterwarnings("ignore", category=DataConversionWarning)

In [7]:
#ip = "10.12.181.191"  # IP address of the receiving device
#port = 5000  # Port number of the receiving device
#client = udp_client.SimpleUDPClient(ip, port)
#address = "/pressure"  # OSC address to send the message to

In [8]:
# Set the duration to stream data (5 seconds in this example)
duration = 5
# Set the sampling rate and channel(s) you want to stream
sampling_rate = 128
channels = list(range(9))  # Streaming data from channels 0 to 7
# Initialize the board
# Create BrainFlowInputParams object and set the parameters
params = BrainFlowInputParams()
params.serial_port = 'COM6'  # Replace with the actual serial port of OpenBCI Cyton board
board = BoardShim(0, params)
board.prepare_session()
while iteraciones < 200:

    # Create empty lists to store the streamed data for each channel
    channel_data = [[] for _ in channels]

    # Start the streaming
    board.start_stream()

    # Get the start time
    start_time = time.time()

    # Loop until the specified duration is reached
    while time.time() - start_time < duration:
        # Fetch the latest available samples
        samples = board.get_current_board_data(sampling_rate)

        # Append the samples to the corresponding channel's data list
        for i, channel in enumerate(channels):
            channel_data[i].extend(samples[channel])

        # Sleep for a small interval to avoid high CPU usage
        time.sleep(0.001)

    # Stop the streaming
    board.stop_stream()

    # Create a dictionary with channel names as keys and data as values
    data_dict = {f'Channel_{channel}': channel_data[i] for i, channel in enumerate(channels)}

    # Create a DataFrame from the data dictionary
    df = pd.DataFrame(data_dict)

    row_all_zeros = (df == 0).all(axis=1)
    df2 = df[~row_all_zeros]
    df3 = df2.drop(df.columns[0], axis=1)
    df4 = df3[['Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5', 'Channel_6', 'Channel_7', 'Channel_8']].copy()

    lowcut = 4  # Lower cutoff frequency in Hz
    highcut = 45  # Upper cutoff frequency in Hz
    fs = 128  # Sampling rate in Hz

    # Apply the bandpass filter to each column
    filtered_df = df4.apply(lambda col: butter_bandpass_filter(col, lowcut, highcut, fs))

    average_reference = filtered_df.mean(axis=1)
    df_average_reference = filtered_df.sub(average_reference, axis=0)

    # Create an empty DataFrame to store the PSD results
    psd_df = pd.DataFrame()

    # Iterate over each column in your DataFrame
    for column in df_average_reference.columns:
        # Compute the PSD for the column data and frequency bands
        psd_bands = compute_psd_bands(df_average_reference[column].values, fs=128)
    
        # Add the PSD values to the DataFrame
        psd_df = psd_df.append(psd_bands, ignore_index=True)

    df_t = psd_df.transpose()
    df_t.columns = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

    df_t = df_t.reset_index()

    # Use the melt function to reshape the DataFrame
    melted_df = pd.melt(df_t, id_vars='index', var_name='channel', value_name='value')

    # Convert channel numbers to strings
    melted_df['channel'] = melted_df['channel'].astype(str)

    # Create a new 'channel_band' column by combining 'channel' and 'index' columns
    melted_df['channel_band'] = melted_df['channel'] + '_' + melted_df['index']

    # Pivot the DataFrame to get the desired format
    new_df = melted_df.pivot(index='index', columns='channel_band', values='value')

    series = new_df.stack()

    # Convert the Series back to a DataFrame with a single row
    filter_df = pd.DataFrame(series)

    valo =filter_df[0]
    valores = valo.reset_index(drop=True)
    df_modelo = pd.DataFrame(valores).transpose()

    df_modelo.columns = ['C3_Alpha', 'C4_Alpha', 'Fp1_Alpha','Fp2_Alpha','O1_Alpha','O2_Alpha','P7_Alpha','P8_Alpha',
                     'C3_Beta', 'C4_Beta', 'Fp1_Beta','Fp2_Beta','O1_Beta','O2_Beta','P7_Beta','P8_Beta',
                     'C3_Delta', 'C4_Delta', 'Fp1_Delta','Fp2_Delta','O1_Delta','O2_Delta','P7_Delta','P8_Delta',
                     'C3_Gamma', 'C4_Gamma', 'Fp1_Gamma','Fp2_Gamma','O1_Gamma','O2_Gamma','P7_Gamma','P8_Gamma',
                     'C3_Theta', 'C4_Theta', 'Fp1_Theta','Fp2_Theta','O1_Theta','O2_Theta','P7_Theta','P8_Theta']
    df_pred = df_modelo.reset_index(drop=True)

    cols = ['Fp1_Delta', 'Fp1_Theta', 'Fp1_Alpha','Fp1_Beta','Fp1_Gamma',
        'Fp2_Delta', 'Fp2_Theta', 'Fp2_Alpha','Fp2_Beta','Fp2_Gamma',
        'C3_Delta', 'C3_Theta', 'C3_Alpha','C3_Beta','C3_Gamma',
        'C4_Delta', 'C4_Theta', 'C4_Alpha','C4_Beta','C4_Gamma',
        'P7_Delta', 'P7_Theta', 'P7_Alpha','P7_Beta','P7_Gamma',
        'P8_Delta', 'P8_Theta', 'P8_Alpha','P8_Beta','P8_Gamma',
        'O1_Delta', 'O1_Theta', 'O1_Alpha','O1_Beta','O1_Gamma',
        'O2_Delta', 'O2_Theta', 'O2_Alpha','O2_Beta','O2_Gamma',] #'C4_Delta', 'C4_Theta', 'C4_Alpha','C4_Beta','C4_Gamma',
    
    df_pred['engagement'] = df_pred['Fp1_Beta'] / (df_pred['Fp1_Theta'] + df_pred['Fp1_Alpha']) 

    df_predi = df_pred[cols]*0.0000488
    iteraciones + 1
    valen = Val_Pkl.predict(df_predi)
    arous = Aro_Pkl.predict(df_predi)
    domin = Dom_Pkl.predict(df_predi)
    vale = mean(valen)
    arou = mean(arous)
    domi = mean(domin)
    emociones = emocion(vale, arou, domi)
    #client.send_message(address, df_pred['engagement'].mean())

    print(emociones)

BrainFlowError: UNABLE_TO_OPEN_PORT_ERROR:2 unable to prepare streaming session